<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/2.0_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salary and Education in Wisconsin - 2.0 Load Landing BigQuery

This notebook is intended to describe analysis on salaries of teachers within the Wisconsin Department of Public Instruction.

## Introduction

### Authenticate to Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()

### Load libraries
Install the following packages in order to load data to BigQuery.

*Please note this will require a restart to the runtime*

In [1]:
!pip install --upgrade google-cloud-bigquery
!pip install gcsfs
!pip install pandas-gbq -U

     |████████████████████████████████| 133kB 4.9MB/s 
  Found existing installation: google-cloud-bigquery 1.8.1
    Uninstalling google-cloud-bigquery-1.8.1:
      Successfully uninstalled google-cloud-bigquery-1.8.1


In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 5)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery

import requests

In [0]:
%matplotlib inline
plt.style.use('bmh')

### Functions

In [0]:
def download_file(url, filename):
  r = requests.get(url)
  f = open(filename,'wb')
  f.write(r.content)
  f.close()

In [0]:
def create_dataset(client, project_id, dataset_name):
  
  
  dataset_id = "{}.{}".format(project_id, dataset_name)
  dataset = bigquery.Dataset(dataset_id)
  dataset.location = "US"

  dataset = client.create_dataset(dataset)
  #print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

### Define Values

In [0]:
download_file("https://raw.githubusercontent.com/j-buss/wi-dpi-analysis/development/eda/wi_dpi_utils.py", "wi_dpi_utils.py")

In [0]:
import wi_dpi_utils as utils

In [0]:
file_dict = [
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/95staff.txt",\
		'new_name':"all_staff_report/1995_1996/95staff.txt",'landing_tablename':"1995",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/96staff.txt",\
		'new_name':"all_staff_report/1996_1997/96staff.txt",'landing_tablename':"1996",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/97staff.txt",\
		'new_name':"all_staff_report/1997_1998/97staff.txt",'landing_tablename':"1997",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/98staff.txt",\
		'new_name':"all_staff_report/1998_1999/98staff.txt",'landing_tablename':"1998",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/99STAFF.DAT",\
		'new_name':"all_staff_report/1999_2000/99STAFF.DAT",'landing_tablename':"1999",'file_type':"fixed"},
    
    
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/00staff.dat",\
		'new_name':"all_staff_report/2000_2001/00staff.dat",'landing_tablename':"2000"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/01staff.dat",\
		'new_name':"all_staff_report/2001_2002/01staff.dat",'landing_tablename':"2001"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/02staff.txt",\
		'new_name':"all_staff_report/2002_2003/02staff.txt",'landing_tablename':"2002"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/03staff.txt",\
		'new_name':"all_staff_report/2003_2004/03staff.txt",'landing_tablename':"2003"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/04staff.dat",\
		'new_name':"all_staff_report/2004_2005/04staff.dat",'landing_tablename':"2004"},
                                                                                                                                                 
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/05staff.txt",\
		'new_name':"all_staff_report/2005_2006/05staff.txt",'landing_tablename':"2005"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/06staff.txt",\
		'new_name':"all_staff_report/2006_2007/06staff.txt",'landing_tablename':"2006"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/07staff.txt",\
		'new_name':"all_staff_report/2007_2008/07staff.txt",'landing_tablename':"2007"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/08STAFF.TXT",\
		'new_name':"all_staff_report/2008_2009/08STAFF.TXT",'landing_tablename':"2008"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/09STAFF.TXT",\
		'new_name':"all_staff_report/2009_2010/09STAFF.TXT",'landing_tablename':"2009"},
                                                                                                                                                 
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/10STAFF.TXT",\
		'new_name':"all_staff_report/2010_2011/10STAFF.TXT",'landing_tablename':"2010"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/11STAFF.txt",\
		'new_name':"all_staff_report/2011_2012/11STAFF.txt",'landing_tablename':"2011"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/12STAFF.txt",\
		'new_name':"all_staff_report/2012_2013/12STAFF.txt",'landing_tablename':"2012"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/13staff.txt",\
		'new_name':"all_staff_report/2013_2014/13staff.txt",'landing_tablename':"2013"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/14staff.txt",\
		'new_name':"all_staff_report/2014_2015/14staff.txt",'landing_tablename':"2014"},
    
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/2015.csv",\
		'new_name':"all_staff_report/2015_2016/2015.csv",'landing_tablename':"2015"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/2016.csv",\
		'new_name':"all_staff_report/2016_2017/2016.csv",'landing_tablename':"2016"}
      
]

In [0]:
project_id='wi-dpi-010'
raw_data_bucket_name='landing-009'

landing_dataset_name='landing'
refined_dataset_name='refined'

## Data Preparation

### Create Dataset

In [0]:
from google.cloud import bigquery

In [0]:
bq_client = bigquery.Client(project=project_id)
create_dataset(bq_client, project_id, "landing")

### Create Tables

In [0]:
from google.cloud import storage

In [0]:
i = file_dict[0]

In [0]:
storage_client = storage.Client(project=project_id)
bucket = storage_client.get_bucket(raw_data_bucket_name)
blob = bucket.get_blob(i['new_name'])
data = blob.download_as_string()

In [24]:
print(data[0:1000])

b'000016044WILSON              JUDITH          FW1962419851995R                                    01 000000003300009888409125264N0802695H  001    NON-EDUCATIONAL AGENCY        NON-EDUCATIONAL AGENCY                                           33                                                                                                                                                                                                     WI                           WI\r\n000003685STARK               LISA                       1995R                                    01 00000070030999630003E120464N1000070OT 001    ALGOMA SCH DIST               ALGOMA SCH DIST               00731KEWAUNEE COUNTY               031715 DIVISION ST              ALGOMA WI  54201                                            1715 DIVISION ST              ALGOMA WI  54201                                            ALGOMA           WI54201     ALGOMA           WI54201     414-487-7001000780DALE N LARSON\r\n000014696

### Load Data

In [0]:
import gcsfs
client = bigquery.Client(project_id)

**Load data from Google Cloud Storage Bucket to BigQuery Landing**

In [39]:
create_dataset(client, project_id, landing_dataset_name)

AttributeError: ignored

In [0]:
print (full_filename)

landing-009/all_staff_report/2017_2018/AllStaffReportPublic__04152019_194414.csv


In [0]:
fs = gcsfs.GCSFileSystem(project=project_id)


In [0]:
i = file_dict[0]

In [0]:
storage_client = storage.Client(project=project_id)
bucket = storage_client.get_bucket(raw_data_bucket_name)
blob = bucket.get_blob(i['new_name'])
data = blob.download_as_string()

In [0]:
print (data[0:1000])

b'000016044WILSON              JUDITH          FW1962419851995R                                    01 000000003300009888409125264N0802695H  001    NON-EDUCATIONAL AGENCY        NON-EDUCATIONAL AGENCY                                           33                                                                                                                                                                                                     WI                           WI\r\n000003685STARK               LISA                       1995R                                    01 00000070030999630003E120464N1000070OT 001    ALGOMA SCH DIST               ALGOMA SCH DIST               00731KEWAUNEE COUNTY               031715 DIVISION ST              ALGOMA WI  54201                                            1715 DIVISION ST              ALGOMA WI  54201                                            ALGOMA           WI54201     ALGOMA           WI54201     414-487-7001000780DALE N LARSON\r\n000014696

In [0]:
df = pd.read_fwf(data, colspecs='infer')

TypeError: ignored

In [0]:
fn = raw_data_bucket_name + '/' + i['new_name']
print(fn)

landing-009/all_staff_report/1995_1996/95staff.txt


In [0]:
with fs.open(fn) as f:
    #print(f.read())
    df = pd.read_fwf(f.read(), colspecs='infer')
    #df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    #df.to_gbq('landing.' + i['landing_tablename'],project_id=project_id,if_exists='replace')
  

TypeError: ignored

In [0]:
for i in file_dict:
  with fs.open(i['new_name']) as f:
    df = pd.read_csv(f, skiprows=1)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.to_gbq('landing.' + i['landing_tablename'],project_id=project_id,if_exists='replace')

FileNotFoundError: ignored

In [0]:

with fs.open(full_filename) as f:
  df = pd.read_csv(f, skiprows=1)

In [0]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.to_gbq(landing_bq_fullname,project_id=project_id,if_exists='replace')